Modul zur Konvertierung von HTML nach PDF installieren

In [2]:
!pip install xhtml2pdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.9/913.9 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.0/447.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 3.3 MB/s eta 0:00:00
  Created wheel for svglib: filename=svglib-1.5.1-py3-none-any.whl size=30906 sha256=0d915ba4f0723370a5e1b1fa564ea19313d09674b3a333063d5a5361250f8d72
  Stored i

JSON Charakter Datei laden

In [42]:
import json
import re
import textwrap
from os import listdir
from os.path import isfile, join
from xhtml2pdf import pisa

TAG_RE = re.compile(r'<[^>]+>')

class DSAEquipmentData():
  name = None
  equipment_type = None
  quantity = None
  talentValue = None
  characteristic1 = None
  characteristic2 = None
  characteristic3 = None
  maxRank = None
  APValue = None
  weight = None
  description = None

  def __init__(self, name, equipment_type):
    self.name = name
    self.equipment_type = equipment_type
    self.quantity = 0
    self.talentValue = 0
    self.characteristic1 = 0
    self.characteristic2 = 0
    self.characteristic3 = 0
    self.maxRank = 0
    self.APValue = 0
    self.weight = 0
    self.description = "n/a"

  def setDescription(self, description):
    self.description = "n/a" if description is None or len(description.strip()) < 1 else '<br/>\n'.join(textwrap.wrap(TAG_RE.sub('', description), width=80))

  def set(self, data_node_system):
    if "description" in data_node_system:
      self.setDescription(data_node_system["description"]["value"])

    self.quantity = data_node_system["quantity"]["value"] if "quantity" in data_node_system else 0
    self.talentValue = data_node_system["talentValue"]["value"] if "talentValue" in data_node_system else 0
    self.characteristic1 = data_node_system["characteristic1"]["value"] if "characteristic1" in data_node_system else None
    self.characteristic2 = data_node_system["characteristic2"]["value"] if "characteristic2" in data_node_system else None
    self.characteristic3 = data_node_system["characteristic3"]["value"] if "characteristic3" in data_node_system else None
    self.maxRank = data_node_system["maxRank"]["value"] if "maxRank" in data_node_system else 0
    self.APValue = data_node_system["APValue"]["value"] if "APValue" in data_node_system else 0
    self.weight = data_node_system["weight"]["value"] if "weight" in data_node_system else 0

  def points(self):
    match self.equipment_type:
      case "spell" | "skill" | "combatskill":
        return self.talentValue

      case "specialability":
        return self.maxRank

      case "advantage" | "disadvantage":
        return self.APValue

      case _:
        return self.quantity

  def characteristics(self, dsa_char):
    return "{} ({}) {} ({}) {} ({})".format(self.characteristic1, dsa_char.getCharacteristic(self.characteristic1),
                                            self.characteristic2, dsa_char.getCharacteristic(self.characteristic2),
                                            self.characteristic3, dsa_char.getCharacteristic(self.characteristic3)).upper()

class DSAChar():
  name = None
  details = None
  life_points = None
  astralenergy = None
  fate_points = None
  experience_total = None
  experience_spent = None
  characteristics = None
  char_items = None

  def __init__(self,json_file):
    self.name = ""
    self.details = {}
    self.life_points = 0
    self.astralenergy = 0
    self.fate_points = 0
    self.experience_total = 0
    self.experience_spent = 0
    self.characteristics = {}
    self.char_items = {}

    self.load(json_file)

  def getCharacteristic(self, characteristic):
    return self.characteristics["CHAR." + characteristic.upper()]

  def load(self,json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    #print(data)
    self.name = data["name"]

    self.life_points = data["system"]["status"]["wounds"]["value"]
    self.astralenergy = data["system"]["status"]["astralenergy"]["value"]
    self.fate_points = data["system"]["status"]["fatePoints"]["value"]

    self.experience_total = data["system"]["details"]["experience"]["total"]
    self.experience_spent = data["system"]["details"]["experience"]["spent"]

    for key,value in data["system"]["details"].items():
        if "value" in value:
            self.details[key] = '<br/>\n'.join(textwrap.wrap(TAG_RE.sub('', value["value"]), width=80))
            #dsa_char.details[key] = textwrap.shorten(TAG_RE.sub('', value["value"]), width=40)
            if len(self.details[key]) < 1:
                self.details[key] = "n/a"

    for key,value in data["system"]["characteristics"].items():
        self.characteristics[value["label"]] = value["initial"] + value["advances"]

    for item in data["items"]:
        item_name = item["name"]
        item_type = item["type"]
        if item_type not in self.char_items:
            self.char_items[item_type] = {}

        self.char_items[item_type][item_name] = DSAEquipmentData(item_name, item_type)
        self.char_items[item_type][item_name].set(item["system"])

    #print(self.name)
    #print(self.details)
    #print(self.characteristics)
    #print(self.char_items)
    #for key,value in self.char_items.items():
    #  for key2,value2 in value.items():
    #    print(value2.equipment_type + "/" + str(value2.points()) + "/" + value2.description)


In [43]:
class DSALevelUpHelper():
  def create(dsa_char: DSAChar):
    characteristic_count = {}

    if "spell" in dsa_char.char_items:
      for spell,spell_data in dsa_char.char_items["spell"].items():
        ch1 = spell_data.characteristic1.upper()
        ch2 = spell_data.characteristic2.upper()
        ch3 = spell_data.characteristic3.upper()

        if ch1 not in characteristic_count:
          characteristic_count[ch1] = []
        if ch2 not in characteristic_count:
          characteristic_count[ch2] = []
        if ch3 not in characteristic_count:
          characteristic_count[ch3] = []

        characteristic_count[ch1].append(spell)
        characteristic_count[ch2].append(spell)
        characteristic_count[ch3].append(spell)

    return characteristic_count


In [45]:
class DSACharTableHelper():
  @staticmethod
  def createTable(dsa_char, item_type, items):
    html_content = '''\n<h2>{}</h2>\n'''.format(item_type)
    html_content = html_content + '''<table border="1" style="border: 1px solid;">\n'''

    match item_type:
      case "spell":
        html_content = html_content + '''<tr>'''
        html_content = html_content + '''<th>Zauberspruch</th>'''
        html_content = html_content + '''<th>ASP</th>'''
        html_content = html_content + '''<th>Talente</th>'''
        html_content = html_content + '''<th>Beschreibung</th>'''
        html_content = html_content + '''</tr>\n'''
        for key,value in items.items():
            html_content = html_content + '''<tr>'''
            html_content = html_content + '''<td class=\"td_left\">{}</td>'''.format(key)
            html_content = html_content + '''<td class=\"td_value\">{}</td>'''.format(value.points())
            html_content = html_content + '''<td class=\"td_value_la_narrow\">{}</td>'''.format(value.characteristics(dsa_char))
            html_content = html_content + '''<td class=\"td_description\">{}</td>'''.format(value.description)
            html_content = html_content + '''</tr>\n'''

      case "skill":
        html_content = html_content + '''<tr>'''
        html_content = html_content + '''<th>Fähigkeit</th>'''
        html_content = html_content + '''<th>Wert</th>'''
        html_content = html_content + '''<th>Talente</th>'''
        html_content = html_content + '''<th>Beschreibung</th>'''
        html_content = html_content + '''</tr>\n'''
        for key,value in items.items():
            html_content = html_content + '''<tr>'''
            html_content = html_content + '''<td class=\"td_left\">{}</td>'''.format(key)
            html_content = html_content + '''<td class=\"td_value\">{}</td>'''.format(value.points())
            html_content = html_content + '''<td class=\"td_value_la\">{}</td>'''.format(value.characteristics(dsa_char))
            html_content = html_content + '''<td class=\"td_description\">{}</td>'''.format(value.description)
            html_content = html_content + '''</tr>\n'''

      case "specialability" | "advantage" | "disadvantage" | "combatskill":
        html_content = html_content + '''<tr>'''
        html_content = html_content + '''<th>Wert</th>'''
        html_content = html_content + '''<th>Punkte</th>'''
        html_content = html_content + '''<th>Beschreibung</th>'''
        html_content = html_content + '''</tr>\n'''
        for key,value in items.items():
            html_content = html_content + '''<tr>'''
            html_content = html_content + '''<td class=\"td_left\">{}</td>'''.format(key)
            html_content = html_content + '''<td class=\"td_value\">{}</td>'''.format(value.points())
            html_content = html_content + '''<td class=\"td_description\">{}</td>'''.format(value.description)
            html_content = html_content + '''</tr>\n'''

      case "money":
        html_content = html_content + '''<tr>'''
        html_content = html_content + '''<th>Dukaten</th>'''
        html_content = html_content + '''<th>Silber</th>'''
        html_content = html_content + '''<th>Heller</th>'''
        if "Kreuzer" in items:
          html_content = html_content + '''<th>Kreuzer</th>'''
        html_content = html_content + '''</tr>\n'''
        html_content = html_content + '''<tr>'''
        html_content = html_content + '''<td class=\"td_value\">{}</td>'''.format(items["Dukaten"].points())
        html_content = html_content + '''<td class=\"td_value\">{}</td>'''.format(items["Silber"].points())
        html_content = html_content + '''<td class=\"td_value\">{}</td>'''.format(items["Heller"].points())
        if "Kreuzer" in items:
          html_content = html_content + '''<td class=\"td_value\">{}</td>'''.format(items["Kreuzer"].points())
        html_content = html_content + '''</tr>\n'''

      case _:
        html_content = html_content + '''<tr>'''
        html_content = html_content + '''<th>Item</th>'''
        html_content = html_content + '''<th>Anzahl</th>'''
        #html_content = html_content + '''<th>Beschreibung</th>'''
        html_content = html_content + '''<th>Gewicht</th>'''
        html_content = html_content + '''</tr>\n'''
        for key,value in items.items():
            html_content = html_content + '''<tr>'''
            html_content = html_content + '''<td class=\"td_left\">{}</td>'''.format(key)
            html_content = html_content + '''<td class=\"td_value\">{}</td>'''.format(value.points())
            #html_content = html_content + '''<td class=\"td_description\">{}</td>'''.format(value.description)
            html_content = html_content + '''<td class=\"td_weight\">{}</td>'''.format(value.weight)
            html_content = html_content + '''</tr>\n'''

    html_content = html_content + '''</table>\n'''

    return html_content

class DSACharPDFCreator():
    def createPDF(dsa_char, out_path):
      # Create HTML for visualization
      html_content = '''<!DOCTYPE html>
              <html>
              <head>
                  <title>''' + dsa_char.name + '''</title>
                  <style>
                    table {border-collapse: collapse;}
                    th {padding: 2px; background-color: #eeeeee; border: 1px solid;}
                    .td_left {min-width: 200px; width: 200px; background-color: #eeeeee; padding: 2px; border: 1px solid; text-align: right; vertical-align:top; font-weight: bold;}
                    .td_right {min-width: 100px; background-color: #ffffff; padding: 2px; border: 1px solid; vertical-align:top;}

                    .td_value {min-width: 50px; width: 50px; background-color: #ffffff; padding: 2px; border: 1px solid; text-align: right; vertical-align:top;}
                    .td_value_la_narrow {min-width: 50px; width: 50px; background-color: #ffffff; padding: 2px; border: 1px solid; text-align: left; vertical-align:top;}
                    .td_value_la {min-width: 150px; width: 150px; background-color: #ffffff; padding: 2px; border: 1px solid; text-align: left; vertical-align:top;}
                    .td_description {min-width: 250px; background-color: #ffffff; padding: 2px; border: 1px solid; vertical-align:top;}
                    .td_weight {min-width: 50px; width: 50px; background-color: #ffffff; padding: 2px; border: 1px solid; text-align: right; vertical-align:top;}
                  </style>
              </head>
              <body>'''

      html_content = html_content + '''<h1>{}</h1>'''.format(dsa_char.name)

      html_content = html_content + '''\n<h2>{}</h2>\n'''.format("Erfahrungspunkte")
      html_content = html_content + '''<table border="1" style="border: 1px solid;">\n'''
      html_content = html_content + '''<tr>'''
      html_content = html_content + '''<td class=\"td_left\">Total - verteilt = übrig</td><td class=\"td_right\">{} - {} = <b>{}</b></td>'''.format(dsa_char.experience_total, dsa_char.experience_spent, dsa_char.experience_total - dsa_char.experience_spent)
      html_content = html_content + '''</tr>\n'''
      html_content = html_content + '''</table>\n'''

      html_content = html_content + '''\n<h2>{}</h2>\n'''.format("Status")
      html_content = html_content + '''<table border="1" style="border: 1px solid;">\n'''
      html_content = html_content + '''<tr><td class=\"td_left\">Lebenspunkte</td><td class=\"td_right\">{}</td></tr>\n'''.format(dsa_char.life_points)
      html_content = html_content + '''<tr><td class=\"td_left\">Astralpunkte</td><td class=\"td_right\">{}</td></tr>\n'''.format(dsa_char.astralenergy)
      html_content = html_content + '''<tr><td class=\"td_left\">Schicksalspunkte</td><td class=\"td_right\">{}</td></tr>\n'''.format(dsa_char.fate_points)
      html_content = html_content + '''</table>\n'''

      html_content = html_content + '''\n<h2>{}</h2>\n'''.format("Details")
      html_content = html_content + '''<table border="1" style="border: 1px solid;">\n'''
      for key, value in dsa_char.details.items():
          html_content = html_content + '''<tr>'''
          html_content = html_content + '''<td class=\"td_left\">{}</td><td class=\"td_right\">{}</td>'''.format(key,value)
          html_content = html_content + '''</tr>\n'''
      html_content = html_content + '''</table>\n'''


      html_content = html_content + '''\n<h2>{}</h2>\n'''.format("Eigenschaften")
      html_content = html_content + '''<table border="1" style="border: 1px solid;">\n'''
      html_content = html_content + '''<tr>'''
      for characteristic_key, characteristic_value in dsa_char.characteristics.items():
          html_content = html_content + '''<th>{}</th>'''.format(characteristic_key)
      html_content = html_content + '''</tr>\n'''
      html_content = html_content + '''<tr>'''
      for characteristic_key, characteristic_value in dsa_char.characteristics.items():
          html_content = html_content + '''<td class=\"td_right\">{}</td>'''.format(characteristic_value)
      html_content = html_content + '''</tr>\n'''
      html_content = html_content + '''</table>\n'''

      for key,value in dsa_char.char_items.items():
          if len(value) < 1:
            continue

          html_content = html_content + DSACharTableHelper.createTable(dsa_char, key, value)

      # Special caalculations
      spell2characteristic_count = DSALevelUpHelper.create(dsa_char)
      if spell2characteristic_count is not None and len(spell2characteristic_count) > 0:
          html_content = html_content + '''\n<h2>{}</h2>\n'''.format("Zauberspruch zu Eigenschaften Mapping")
          html_content = html_content + '''<table border="1" style="border: 1px solid;">\n'''
          for characteristic,spells in spell2characteristic_count.items():
              html_content = html_content + '''<tr>'''
              html_content = html_content + '''<td class=\"td_left\">{} ({})</td>'''.format(characteristic, dsa_char.getCharacteristic(characteristic))
              html_content = html_content + '''<td class=\"td_value\">{}</td>'''.format(len(spells))
              html_content = html_content + '''<td class=\"td_right\">{}</td>'''.format(', '.join(spells))
              html_content = html_content + '''</tr>\n'''
          html_content = html_content + '''</table>\n'''

      html_content = html_content + '''</body></html>'''

      #print(html_content)

      # Convert the generated HTML to PDF
      filename = out_path + "/" + dsa_char.name + ".pdf"
      with open(filename, "wb") as pdf_file:
          pisa_status = pisa.CreatePDF(html_content, dest=pdf_file)
          print(pisa_status)


In [46]:
dsa_char_path = "/content/drive/MyDrive/DSAChars"
out_path = "/content/drive/MyDrive/DSACharsPDF"

print("fvtt_dsa_json2PDF starting...")
for json_file in listdir(dsa_char_path):
  file_path = join(dsa_char_path, json_file)
  if isfile(file_path):
    print("Working on file {}".format(file_path))
    dsa_char = DSAChar(file_path)
    print("Working on DSA char: {}".format(dsa_char.name))
    DSACharPDFCreator.createPDF(dsa_char, out_path)
print("fvtt_dsa_json2PDF finished.")


'<table border="1" style="border: 1px solid;"> </table>'


fvtt_dsa_json2PDF starting...
Working on file /content/drive/MyDrive/DSAChars/fvtt-Actor-madali-WFJnCIIa7WcMskDK.json
Working on DSA char: Madali
Working on file /content/drive/MyDrive/DSAChars/fvtt-Actor-romoxosch-sohn-des-dwarosch-0LQNyCcMAox2n55d.json
Working on DSA char: Romoxosch Sohn des Dwarosch
Working on file /content/drive/MyDrive/DSAChars/fvtt-Actor-quinlan-katzenleben-FZ8c19dKc05PpA42.json
Working on DSA char: Quinlan Katzenleben
Working on file /content/drive/MyDrive/DSAChars/fvtt-Actor-caerleon-fichtenkind-g7s7zuo6XcnxTe9L.json
Working on DSA char: Caerleon Fichtenkind
Working on file /content/drive/MyDrive/DSAChars/fvtt-Actor-leomara-barenherz-Jsb63XAMBfluN1Nf.json
Working on DSA char: Leomara Bärenherz
Working on file /content/drive/MyDrive/DSAChars/fvtt-Actor-hesindian-schlangentreu-pCXAwjkUPBP5X2uz.json
Working on DSA char: Hesindian Schlangentreu
fvtt_dsa_json2PDF finished.
